## STEP 2
## Search the weather for the city list provided by the marketing team

Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

In [1]:
import requests
import pandas as pd
import json
from datetime import datetime, timezone

In [1]:
# api key to connect 
api_key = "YOUR_API_KEY"

In [3]:
#open the city list file
cityGeo = pd.read_csv("refCity.csv")
cityGeo.head()

,CityID,CityName,CityLat,CityLon
0,0,Mont Saint Michel,48.635954,-1.511460
1,1,St Malo,48.649518,-2.026041
2,2,Bayeux,49.276462,-0.702474
3,3,Le Havre,49.493898,0.107973
4,4,Rouen,49.440459,1.093966


In [4]:
cityGeo.isna().sum().sum()

0

In [5]:
columnsToUse = ['pop','feels_like','temp','clouds','wind_speed']
# sorting rule for the selection
sortAscending = [False,False,False,True,True]
dataWeatherFinal = pd.DataFrame(columns = ['From','To','CityID','CityName','CityLat','CityLon'] + columnsToUse)
dataWeatherFinal.head()

,From,To,CityID,CityName,CityLat,CityLon,pop,feels_like,temp,clouds,wind_speed


Search the weather data to each city

In [6]:
for indx, row in cityGeo.iterrows():
    lat=row["CityLat"]
    lon=row["CityLon"]
    url="https://api.openweathermap.org/data/2.5/onecall?lat=%s&lon=%s&exclude=current,hourly,minutely&appid=%s&units=metric" % (lat, lon, api_key)
    response = requests.get(url)
    data = json.loads(response.text)
    dataWeather = pd.DataFrame(data.get('daily'))
    ##print(dataWeather.head())
    dataWeather['temp'] = dataWeather['temp'].apply(lambda x: x.get('day'))
    dataWeather['feels_like'] = dataWeather['feels_like'].apply(lambda x: x.get('day'))
    dateFrom = pd.to_datetime(datetime.fromtimestamp(dataWeather['dt'].min()).isoformat())
    dateTo = pd.to_datetime(datetime.fromtimestamp(dataWeather['dt'].max()).isoformat())    
    dataToInsert = dataWeather[columnsToUse].mean().to_dict()
    dataToInsert.update({'From':dateFrom,'To':dateTo,'CityName':row["CityName"],'CityID':row["CityID"],'CityLat':lat,'CityLon':lon})
    dataWeatherFinal = dataWeatherFinal.append(dataToInsert,ignore_index=True)

In [7]:
print(dataWeatherFinal.shape)
dataWeatherFinal.head()

(35, 11)


,From,To,CityID,CityName,CityLat,CityLon,pop,feels_like,temp,clouds,wind_speed
0,2022-01-29 13:00:00,2022-02-05 13:00:00,0,Mont Saint Michel,48.635954,-1.511460,0.21125,7.80000,9.56875,77.625,6.93125
1,2022-01-29 13:00:00,2022-02-05 13:00:00,1,St Malo,48.649518,-2.026041,0.24625,7.45000,9.38250,78.750,7.84375
2,2022-01-29 13:00:00,2022-02-05 13:00:00,2,Bayeux,49.276462,-0.702474,0.22500,7.30625,9.22250,68.625,7.53500
3,2022-01-29 13:00:00,2022-02-05 13:00:00,3,Le Havre,49.493898,0.107973,0.33375,5.77750,8.54875,69.375,9.00375
4,2022-01-29 13:00:00,2022-02-05 13:00:00,4,Rouen,49.440459,1.093966,0.31375,6.18500,8.72500,70.875,7.22500


In [8]:
#dataWeatherFinal.to_csv("weather_data.csv",index=False)

Search the top 5 cities by best weather

In [9]:
dataWeatherTop5 = dataWeatherFinal.sort_values(by = columnsToUse, ascending = sortAscending).reset_index(drop=True).head(5)

In [10]:
dataWeatherTop5.head()

,From,To,CityID,CityName,CityLat,CityLon,pop,feels_like,temp,clouds,wind_speed
0,2022-01-29 12:00:00,2022-02-05 12:00:00,12,Besancon,47.238022,6.024362,0.59500,3.53875,5.90000,88.500,4.19250
1,2022-01-29 12:00:00,2022-02-05 12:00:00,9,Chateau du Haut Koenigsbourg,48.249490,7.344296,0.58625,-0.38750,2.34000,90.625,3.65125
2,2022-01-29 12:00:00,2022-02-05 12:00:00,11,Eguisheim,48.044797,7.307962,0.58625,4.34750,6.15125,88.250,3.74250
3,2022-01-29 12:00:00,2022-02-05 12:00:00,10,Colmar,48.077752,7.357964,0.58375,4.08000,6.04750,89.375,3.93000
4,2022-01-29 12:00:00,2022-02-05 12:00:00,8,Strasbourg,48.584614,7.750713,0.56375,2.73750,5.65000,85.500,5.54500


In [11]:
dataWeatherTop5.rename(columns={"pop":"Pop","feels_like":"FeelLike","temp":"Temp","clouds":"Clouds" ,"wind_speed":"WindSpeed"},inplace=True)

In [12]:
dataWeatherTop5.columns

Index(['From', 'To', 'CityID', 'CityName', 'CityLat', 'CityLon', 'Pop',
       'FeelLike', 'Temp', 'Clouds', 'WindSpeed'],
      dtype='object')

Save the final data in the csv file

In [13]:
dataWeatherTop5.head(5).to_csv('weather_for_top5_city.csv', index=False)